# -*- coding: utf-8 -*-
"""
Created on Tue Feb 28

@author: Kuldeep Kumar
"""

In [1]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
#from sklearn.preprocessing import
from sklearn.impute import SimpleImputer
import numpy as np
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

Using TensorFlow backend.


load dataset

In [2]:
missing_values = ["n/a", "na", "Nan","--", "?"]
# Importing the Dataset
dataframe = pd.read_csv("heartdisease_data.csv", header=None, na_values=missing_values)

In [3]:
print("Null Valve in dataset ", dataframe.isnull().sum())

Null Valve in dataset  0       0
1       0
2       0
3      56
4       7
5       7
6       0
7      53
8      53
9      56
10    102
11    198
12    166
13      0
dtype: int64


In [4]:
dataframe=dataframe.replace("?", "nan")

In [5]:
for i in dataframe.columns[dataframe.isnull().any(axis=0)]:
    dataframe[i].fillna(dataframe[i].mean(),inplace=True)

In [6]:
print("Null Valve in dataset after impute", dataframe.isnull().sum())

Null Valve in dataset after impute 0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
dtype: int64


In [7]:
dataframe.to_csv("111.csv")

In [8]:
dataset = dataframe.values
X = dataset[:,0:13].astype(float)
Y = dataset[:,13]

convert into binary classification problem - heart disease or no heart disease

In [9]:
Y_binary = Y.copy()
Y_binary[Y_binary > 0] = 1

In [10]:
print("Y Binary",Y_binary)

Y Binary [1. 0. 1. 1. 0. 0. 1. 0. 1. 1. 0. 1. 1. 1. 1. 0. 1. 1. 0. 0. 1. 1. 1. 0.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 0. 1. 1. 1. 0. 1. 1. 1. 0.
 1. 0. 1. 0. 1. 0. 1. 1. 1. 1. 0. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1.
 0. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 0. 1. 1. 0. 1.
 0. 1. 1. 0. 1. 1. 1. 1. 0. 1. 1. 1. 0. 0. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 0. 1. 1. 1. 1. 0. 0. 1. 1. 1. 0. 1. 0. 1. 1. 0. 1. 0. 1. 1. 1.
 0. 0. 0. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0.
 1. 1. 1. 0. 1. 1. 0. 0. 1. 1. 1. 1. 1. 0. 1. 1. 0. 1. 1. 1. 1. 0. 0. 1.
 1. 1. 1. 1. 0. 1. 0. 1.]


define baseline model

In [11]:
def baseline_model():
    model = Sequential()
    model.add(Dense(13, input_dim=13, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [12]:
estimator = KerasClassifier(build_fn=baseline_model, epochs=200, batch_size=5, verbose=0)
kfold = KFold(n_splits=10, shuffle=True)
results = cross_val_score(estimator, X, Y_binary, cv=kfold)
print("Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Accuracy: 76.00% (9.43%)


Output is Accuracy: 72.50% (11.01%)

evaluate baseline model with standardized dataset

In [13]:
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=baseline_model, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True)
results = cross_val_score(pipeline, X, Y_binary, cv=kfold)
print("Standardized: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Standardized: 75.96% (6.73%)


Output Accuracy after standardize of data<br>
tandardized: 75.00% (8.37%)

Evaluvate using large neural network

larger model

In [14]:
def create_larger():
    model = Sequential()
    model.add(Dense(13, input_dim=13, activation='relu'))
    model.add(Dense(30, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [15]:
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_larger, epochs=100, batch_size=5,verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True)
results = cross_val_score(pipeline, X, Y_binary, cv=kfold)
print("Larger: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Larger: 69.98% (8.41%)


Output for larger network<br>
Larger: 69.98% (8.41%)